<a href="https://colab.research.google.com/github/Priyansu171203/AIML-Task-4/blob/main/task_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch gradio --quiet

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import gradio as gr

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.eval()

# Average words per line to estimate tokens length (adjust if you want)
AVG_WORDS_PER_LINE = 10

def generate_text(prompt, num_lines=10):
    try:
        prompt_tokens = len(tokenizer.encode(prompt))
        # Estimate tokens needed for desired lines
        max_new_tokens = num_lines * AVG_WORDS_PER_LINE

        max_length = prompt_tokens + max_new_tokens
        inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)

        output = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            do_sample=True,
            temperature=0.9,
            top_k=50,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id
        )

        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        # Remove prompt from output if present
        if generated_text.startswith(prompt):
            generated_text = generated_text[len(prompt):].strip()

        # Limit output to approx num_lines by splitting on newlines or sentences
        lines = generated_text.split('\n')
        if len(lines) < num_lines:
            # If fewer lines, try splitting by periods as fallback
            lines = generated_text.split('. ')

        limited_output = '\n'.join(lines[:num_lines])
        return limited_output
    except Exception as e:
        return f"[Error]: {e}"


In [ ]:
iface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(lines=5, label="Enter a prompt", placeholder="Once upon a time..."),
        gr.Slider(1, 100, step=1, value=10, label="Number of output lines")
    ],
    outputs=gr.Textbox(lines=20, label="Generated Text"),
    title="🧠 GPT-2 Text Generator by Lines",
    description="Enter a prompt and select how many lines of output you want (1 to 100)."
)

iface.launch()
